In [ ]:
import shapely.geometry
import pandas as pd
import numpy as np
import importlib
import matplotlib.pyplot as plt
from datetime import datetime
import time
import copy

import math
import IPython
import pickle
import os

In [ ]:
#좌표 v가 속하는 그리드의 키값 반환
def find_grid_key(llist, v):
    result = -1
    for i in range(0, len(llist)-1):
        if llist[i] <= v and llist[i+1] >= v:
            return llist[i]
    return result

In [ ]:
def find_grid_idx(llist, v):
    result = -1
    for i in range(0, len(llist)-1):
        if llist[i] <= v and llist[i+1] >= v:
            return i
    return result

#그리드에 도착지와의 유클리드 거리 값 할당

In [ ]:
# 유클리드 거리
def euclid_distance(point_1, point_2):
  x1, y1 = point_1
  x2, y2 = point_2
  distance = np.sqrt((x1-x2)**2 + (y1-y2)**2)
  return distance

In [ ]:
def insert_data_distance(grid_map, grid_lat, grid_lon, grid_lat_lon_keys, destination):
    for i in range(len(grid_lat)):
        for j in range(len(grid_lon)):
            key = '(%f,%f)'%(grid_lat[i], grid_lon[j])
            if key not in grid_lat_lon_keys:
                continue
            distance = euclid_distance((grid_lat[i], grid_lon[j]), destination)
            grid_map[key][7] = distance

# 거리값 퀀타일 적용, 스코어링

In [ ]:

def quantile_distance(grid_map, grid_lat_lon_keys, destination): # (육지 제외 그리드, 도착지)
  distance_values = []
  for key in grid_lat_lon_keys:
    distance = grid_map[key][7]
    distance_values.append(distance)

  ratio = [0.25, 0.5, 0.75]
  quantile_values = np.quantile(distance_values, ratio)
  return quantile_values

def score_distance(grid_map, grid_lat_lon_keys, destination, score_dic):
    quantile_values = quantile_distance(grid_map, grid_lat_lon_keys, destination)
    q_1, q_2, q_3 = quantile_values

    for key in grid_map.keys():
      # 1단계
      if 0 <= grid_map[key][7] < q_1:
        score_dic[key][4] = 1
      # 2단계
      elif q_1 <= grid_map[key][7] < q_2:
        score_dic[key][4] = 2
      # 3단계
      elif q_2 <= grid_map[key][7] < q_3:
        score_dic[key][4] = 3
      # 4단계
      else:
        score_dic[key][4] = 4